In [9]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [22]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

Prompt==> Problem: Write a function in c++ to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-56e44ddeca9648ab988ee4bccc4c6d33","object":"text_completion","created":452713,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nint sumOfDigits(int num) {\n    int sum = 0;\n    while (num != 0) {\n        sum += num % 10;\n        num /= 10;\n    }\n    return sum;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a positive integer: \";\n    cin >> num;\n    cout << \"Sum of digits: \" << sumOfDigits(num) << endl;\n    return 0;\n}\n```\n\nThis program first asks the user to input a positive integer. It then calls the function `sumOfDigits` with the input number as the argument. The function calculates the sum of the digits by re

In [11]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [12]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [13]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [14]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [17]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [20]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-b457750dfa214b34b19846e0f699484a","object":"text_completion","created":452677,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.779312 seconds 


Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-42796ccf5a8b4a5c86ddd6d92c0e51bc","object":"text_completion","created":452677,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-0a2fa777aa57470799e964a160db6552","object":"text_completion","created":452689,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 6.108930 seconds 


Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integ

KeyboardInterrupt: 

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))